# Logical Alarms for Misaligned 

The power of logic alone as an aid for AI safety is demonstrated by the algorithms in the `ntqr.alams` module. This notebook provides some examples demonstrating their use.

## Binary classification, the R=2 case

In [15]:
import ntqr.alarms, ntqr.r2
import matplotlib.pyplot as plt
import sympy, random
sympy.init_session()

IPython console for SymPy 1.13.2 (Python 3.11.8-64-bit) (ground types: gmpy)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.13.2/

